In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# ===== Configurações gerais =====
INPUT_CSV = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"   # seu arquivo
OUTPUT_DIR = "saidas_modelos"
os.makedirs(OUTPUT_DIR, exist_ok=True)
FEATS = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']  # sem macro
TARGET = 'close_plus1'
N_SPLITS = 5  # TimeSeriesSplit
# ===== Funções utilitárias =====
def roll_mean(a, n): return np.mean(a[-n:]) if len(a) >= n else np.nan
def roll_std(a, n):  return np.std(a[-n:], ddof=1) if len(a) >= n else np.nan
def carregar_base():
   df = pd.read_csv(INPUT_CSV, parse_dates=['date'])
   df = df.sort_values('date').reset_index(drop=True)
   assert pd.isna(df.loc[len(df)-1, TARGET]), "A última linha (dez/2023) deve ter close_plus1 em branco."
   return df
def preparar_treino(df):
   df_train = df.dropna(subset=FEATS + [TARGET]).copy()
   X = df_train[FEATS].values
   y = df_train[TARGET].values
   return df_train, X, y
def avaliar_oof(model, X, y, scaler=False):
   """Avalia com TimeSeriesSplit e retorna métricas + previsões OOF."""
   tscv = TimeSeriesSplit(n_splits=N_SPLITS)
   y_oof = np.empty_like(y, dtype=float); y_oof[:] = np.nan
   for tr, te in tscv.split(X):
       model.fit(X[tr], y[tr])
       y_oof[te] = model.predict(X[te])
   mae  = mean_absolute_error(y, y_oof)
   rmse = mean_squared_error(y, y_oof, squared=False)
   r2   = r2_score(y, y_oof)
   return {'MAE': mae, 'RMSE': rmse, 'R2': r2}, y_oof
def treinar_escolher(model, grid, X, y):
   """GridSearchCV com TimeSeriesSplit; retorna melhor estimador e params."""
   tscv = TimeSeriesSplit(n_splits=N_SPLITS)
   g = GridSearchCV(model, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
   g.fit(X, y)
   best = g.best_estimator_
   return best, g.best_params_
def projetar_2024(model, df_hist):
   """
   Projeção recursiva jan–dez/2024:
   - usa apenas features internas
   - recalcula MM/Vol/Momentum a cada mês com closes previstos
   - volrel futuro: média dos últimos 3 observados
   """
   df_work = df_hist[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
   volrel_last3 = df_work['volrel'].tail(3).mean()
   last_close = df_work['close'].iloc[-1]
   rows = []
   for i in range(12):
       feats_now = df_work.iloc[-1][FEATS].to_dict()
       feats_now['volrel'] = volrel_last3  # suposição p/ o mês seguinte
       x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
       pred_close_next = float(model.predict(x)[0])
       next_date = df_work['date'].iloc[-1] + pd.offsets.MonthBegin(1)
       ret_next = (pred_close_next / last_close) - 1.0
       # anexa linha prevista
       row = {'date': next_date,
              'close': pred_close_next,
              'ret_m': ret_next,
              'volrel': volrel_last3}
       df_work = pd.concat([df_work, pd.DataFrame([row])], ignore_index=True)
       # recalcula features na última linha
       closes = df_work['close'].values
       rets   = df_work['ret_m'].values
       df_work.loc[df_work.index[-1], 'mm3']  = roll_mean(closes, 3)
       df_work.loc[df_work.index[-1], 'mm6']  = roll_mean(closes, 6)
       df_work.loc[df_work.index[-1], 'mm12'] = roll_mean(closes, 12)
       df_work.loc[df_work.index[-1], 'vol3'] = roll_std(rets, 3)
       df_work.loc[df_work.index[-1], 'vol6'] = roll_std(rets, 6)
       if len(closes) >= 4:
           df_work.loc[df_work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
       last_close = pred_close_next
       rows.append({'date': next_date, 'close_pred': pred_close_next})
   return pd.DataFrame(rows)
def salvar_excel(caminho, previsoes_df, metricas, best_params):
   with pd.ExcelWriter(caminho, engine='xlsxwriter') as writer:
       previsoes_df.to_excel(writer, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([metricas]).to_excel(writer, index=False, sheet_name='metricas')
       pd.DataFrame([best_params]).to_excel(writer, index=False, sheet_name='best_params')
# =======================
# 1) REGRESSÃO LINEAR (RIDGE)
# =======================
df = carregar_base()
df_train, X, y = preparar_treino(df)
ridge = Pipeline([('scaler', StandardScaler()), ('model', Ridge())])
grid_ridge = {'model__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
best_ridge, bestp_ridge = treinar_escolher(ridge, grid_ridge, X, y)
met_ridge, _ = avaliar_oof(best_ridge, X, y)
pred_ridge = projetar_2024(best_ridge, df)
salvar_excel(os.path.join(OUTPUT_DIR, 'bova11_pred_2024_ridge.xlsx'),
            pred_ridge, met_ridge, bestp_ridge)
print("Arquivo salvo:", os.path.join(OUTPUT_DIR, 'bova11_pred_2024_ridge.xlsx'))
# =======================
# 2) ÁRVORE DE DECISÃO
# =======================
tree = DecisionTreeRegressor(random_state=42)
grid_tree = {'max_depth': [4, 6, 8, 10],
            'min_samples_leaf': [5, 10, 20]}
best_tree, bestp_tree = treinar_escolher(tree, grid_tree, X, y)
met_tree, _ = avaliar_oof(best_tree, X, y)
pred_tree = projetar_2024(best_tree, df)
salvar_excel(os.path.join(OUTPUT_DIR, 'bova11_pred_2024_tree.xlsx'),
            pred_tree, met_tree, bestp_tree)
print("Arquivo salvo:", os.path.join(OUTPUT_DIR, 'bova11_pred_2024_tree.xlsx'))
# =======================
# 3) RANDOM FOREST
# =======================
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_rf = {'n_estimators': [300, 500],
          'max_depth': [6, 8, 12],
          'min_samples_leaf': [3, 5, 10],
          'max_features': [0.6, 'sqrt']}
best_rf, bestp_rf = treinar_escolher(rf, grid_rf, X, y)
met_rf, _ = avaliar_oof(best_rf, X, y)
pred_rf = projetar_2024(best_rf, df)
salvar_excel(os.path.join(OUTPUT_DIR, 'bova11_pred_2024_rf.xlsx'),
            pred_rf, met_rf, bestp_rf)
print("Arquivo salvo:", os.path.join(OUTPUT_DIR, 'bova11_pred_2024_rf.xlsx'))

ValueError: Missing column provided to 'parse_dates': 'date'

In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv", sep=";")   # ou o caminho completo
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")
print(df.head(49))
# === 0) PADRONIZAÇÃO NUMÉRICA (se seu CSV veio com vírgula decimal) ===
# Se suas colunas já estão numéricas, este bloco não fará nada de errado.
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str).str.replace('.', '', regex=False)  # remove separador de milhar, se houver
                          .str.replace(',', '.', regex=False))       # vírgula -> ponto
       df[c] = pd.to_numeric(df[c], errors='coerce')
# Conferir última linha do alvo (DEVE estar vazia para prever jan/2024)
assert pd.isna(df.loc[len(df)-1, 'close_plus1']), "A última linha (dez/2023) deve estar vazia em close_plus1."
# === 1) TREINO (Ridge) com validação temporal ===
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
FEATS = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
# tira linhas iniciais com janelas incompletas e a última (alvo vazio)
train = df.dropna(subset=FEATS + [TARGET]).copy()
X = train[FEATS].values
y = train[TARGET].values
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
tscv  = TimeSeriesSplit(n_splits=5)
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# OOF metrics
y_oof = np.empty_like(y, dtype=float); y_oof[:] = np.nan
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mae  = mean_absolute_error(y, y_oof)
rmse = mean_squared_error(y, y_oof, squared=False)
r2   = r2_score(y, y_oof)
print(f"Ridge  | MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# === 2) PROJEÇÃO RECURSIVA (JAN–DEZ/2024) ===
def roll_mean(a, n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a, n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
work = df[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close    = work['close'].iloc[-1]
rows = []
for i in range(12):
   # features do mês t (última linha disponível) para prever t+1
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3  # suposição para o próximo mês
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   next_date = work['date'].iloc[-1] + pd.offsets.MonthBegin(1)
   ret_next  = (pred_close_next / last_close) - 1.0
   # anexa linha prevista
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   # recalcula features na última linha
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_ridge = pd.DataFrame(rows)
print(pred_ridge)
# === 3) SALVAR EM EXCEL ===
with pd.ExcelWriter('bova11_pred_2024_ridge.xlsx', engine='xlsxwriter') as w:
   pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
   pd.DataFrame([{'MAE':mae, 'RMSE':rmse, 'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
print("Arquivo gerado: bova11_pred_2024_ridge.xlsx")

         date   close    ret_m       mm3       mm6      mm12    vol3    vol6  \
0  2020-01-01   108,9  -0,0209    108,16  104,3883   99,2958  0,0614  0,0614   
1  2020-02-01   100,6  -0,0762    106,91  104,8867   100,025  0,0717  0,0717   
2  2020-03-01   69,35  -0,3106     92,95     99,61   98,1533  0,1538  0,1611   
3  2020-04-01   77,21   0,1133   82,3867   95,2733   96,8475  0,2124  0,1654   
4  2020-05-01   84,15   0,0899   76,9033   91,9067   96,0608  0,2383   0,158   
5  2020-06-01   91,62   0,0888   84,3267   88,6383   95,6033  0,0139  0,1608   
6  2020-07-01   99,29   0,0837   91,6867   87,0367   95,7125  0,0033  0,1662   
7  2020-08-01    95,7  -0,0362   95,5367     86,22   95,5533  0,0707  0,1634   
8  2020-09-01   91,05  -0,0486   95,3467   89,8367   94,7233  0,0731  0,0712   
9  2020-10-01   90,66  -0,0043     92,47   92,0783   93,6758  0,0229  0,0658   
10 2020-11-01     105   0,1582     95,57   95,5533     93,73  0,1089  0,0823   
11 2020-12-01  114,65   0,0919  103,4367

ValueError: Input contains NaN.

In [2]:
print("NaN por coluna:\n", df[FEATS + [TARGET]].isna().sum())
print("Linhas com NaN nas features/target:")
print(df[df[FEATS + [TARGET]].isna().any(1)].tail(10))

NaN por coluna:
 ret_m          0
mm3            0
mm6            0
mm12           0
vol3           0
vol6           0
momentum3      0
volrel         0
close_plus1    1
dtype: int64
Linhas com NaN nas features/target:
         date   close   ret_m       mm3       mm6     mm12    vol3    vol6  \
47 2023-12-01  130.39  0.0552  121.1933  117.8317  111.365  0.0793  0.0636   

    momentum3  volrel selic_m  ipca_m  close_plus1  
47     0.1524  0.7966  0,1175  0,0056          NaN  


In [3]:
import pandas as pd
df = pd.read_csv(r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv", sep=";")   # ou o caminho completo
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")
print(df.head(49))
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
FEATS = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
# 0) Garante numérico (se já estiver, não muda nada)
for c in ['close'] + FEATS + [TARGET]:
   df[c] = pd.to_numeric(df[c], errors='coerce')
# 1) Treino: remove linhas sem janelas completas e a última (TARGET vazio)
train = df.dropna(subset=FEATS + [TARGET]).copy()
X = train[FEATS].values
y = train[TARGET].values
# 2) Modelo + busca de alpha
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
tscv  = TimeSeriesSplit(n_splits=5)
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# 3) OOF com máscara (evita erro por NaN)
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
rmse = mean_squared_error(y[mask], y_oof[mask], squared=False)
r2   = r2_score(y[mask], y_oof[mask])
print(f"Ridge  | MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# 4) Projeção recursiva JAN–DEZ/2024 (sem macro)
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
work = df[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
rows = []
for _ in range(12):
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3  # suposição para o mês seguinte
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   next_date = work['date'].iloc[-1] + pd.offsets.MonthBegin(1)
   ret_next  = (pred_close_next / last_close) - 1.0
   # anexa linha prevista
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   # recalcula features na última linha
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_ridge = pd.DataFrame(rows)
# 5) Salvar em Excel
with pd.ExcelWriter('bova11_pred_2024_ridge.xlsx', engine='xlsxwriter') as w:
   pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
   pd.DataFrame([{'MAE':mae, 'RMSE':rmse, 'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
print("Arquivo gerado: bova11_pred_2024_ridge.xlsx")

         date   close    ret_m       mm3       mm6      mm12    vol3    vol6  \
0  2020-01-01   108,9  -0,0209    108,16  104,3883   99,2958  0,0614  0,0614   
1  2020-02-01   100,6  -0,0762    106,91  104,8867   100,025  0,0717  0,0717   
2  2020-03-01   69,35  -0,3106     92,95     99,61   98,1533  0,1538  0,1611   
3  2020-04-01   77,21   0,1133   82,3867   95,2733   96,8475  0,2124  0,1654   
4  2020-05-01   84,15   0,0899   76,9033   91,9067   96,0608  0,2383   0,158   
5  2020-06-01   91,62   0,0888   84,3267   88,6383   95,6033  0,0139  0,1608   
6  2020-07-01   99,29   0,0837   91,6867   87,0367   95,7125  0,0033  0,1662   
7  2020-08-01    95,7  -0,0362   95,5367     86,22   95,5533  0,0707  0,1634   
8  2020-09-01   91,05  -0,0486   95,3467   89,8367   94,7233  0,0731  0,0712   
9  2020-10-01   90,66  -0,0043     92,47   92,0783   93,6758  0,0229  0,0658   
10 2020-11-01     105   0,1582     95,57   95,5533     93,73  0,1089  0,0823   
11 2020-12-01  114,65   0,0919  103,4367

ValueError: Cannot have number of folds=6 greater than the number of samples=0.

In [4]:
print("Total de linhas no df:", len(df))
print("Total de linhas após dropna:", len(df.dropna(subset=FEATS + [TARGET])))
print(df[FEATS + [TARGET]].isna().sum())

Total de linhas no df: 48
Total de linhas após dropna: 0
ret_m          48
mm3            48
mm6            48
mm12           48
vol3           48
vol6           48
momentum3      48
volrel         48
close_plus1    46
dtype: int64


In [5]:
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# 1) Ler o CSV (usa sep=";", decimal="," e milhares=".")
df = pd.read_csv(
   r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv",
   sep=";", decimal=",", thousands="."
)
# 2) Normalizar headers
df.columns = (df.columns
               .str.strip()
               .str.replace(r"\s+", "_", regex=True)
               .str.lower())
# 3) Renomear para os nomes usados no modelo (ajuste se seu header estiver diferente)
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',  # caso venha "Último"
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m', 'ret_m': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
# 4) Datas
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
# 5) Limpeza e conversão numérica robusta
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                      .str.replace('\u00A0', '', regex=False)  # espaço não-quebrante
                      .str.replace('%', '', regex=False)
                      .str.replace(' ', '', regex=False))
       # Se seu CSV já veio com decimal=",", thousands="." isso normalmente basta:
       df[c] = pd.to_numeric(df[c], errors='coerce')
# 6) Checagens
print("Dtypes:\n", df[num_cols].dtypes)
print("\nNaN por coluna:\n", df[num_cols].isna().sum())
assert pd.isna(df.loc[len(df)-1, 'close_plus1']), "A última linha (dez/2023) deve estar vazia em close_plus1."
# 7) Montar treino
FEATS  = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
train = df.dropna(subset=FEATS + [TARGET]).copy()
print("\nLinhas totais:", len(df), " | Linhas após dropna:", len(train))
assert len(train) >= 12, "Poucos dados após limpeza; verifique colunas renomeadas e conversão numérica."
X = train[FEATS].values
y = train[TARGET].values
# 8) Ridge com OOF
tscv  = TimeSeriesSplit(n_splits=5)
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
rmse = mean_squared_error(y[mask], y_oof[mask], squared=False)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge | MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# 9) Projeção recursiva JAN–DEZ/2024
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
work = df[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
rows = []
for _ in range(12):
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1,-1)
   pred_close_next = float(best_ridge.predict(x)[0])
   next_date = work['date'].iloc[-1] + pd.offsets.MonthBegin(1)
   ret_next  = (pred_close_next / last_close) - 1.0
   # anexa a nova linha e recalcula features
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1],'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1],'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1],'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1],'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1],'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1],'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_ridge = pd.DataFrame(rows)
# 10) Salvar Excel
try:
   with pd.ExcelWriter('bova11_pred_2024_ridge.xlsx', engine='xlsxwriter') as w:
       pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE':mae,'RMSE':rmse,'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
except Exception:
   with pd.ExcelWriter('bova11_pred_2024_ridge.xlsx', engine='openpyxl') as w:
       pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE':mae,'RMSE':rmse,'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
print("\nArquivo gerado: bova11_pred_2024_ridge.xlsx")

Dtypes:
 close          float64
ret_m          float64
mm3            float64
mm6            float64
mm12           float64
vol3           float64
vol6           float64
momentum3      float64
volrel         float64
close_plus1    float64
dtype: object

NaN por coluna:
 close          0
ret_m          0
mm3            0
mm6            0
mm12           0
vol3           0
vol6           0
momentum3      0
volrel         0
close_plus1    1
dtype: int64

Linhas totais: 48  | Linhas após dropna: 47


TypeError: got an unexpected keyword argument 'squared'

In [6]:
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# 1) Ler CSV (ajuste o caminho conforme necessário)
df = pd.read_csv(
   r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv",
   sep=";", decimal=",", thousands="."
)
# 2) Normalizar cabeçalhos
df.columns = (df.columns
               .str.strip()
               .str.replace(r"\s+", "_", regex=True)
               .str.lower())
# 3) Renomear colunas conforme padrão
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
# 4) Converter datas
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
# 5) Converter colunas numéricas (limpando caracteres)
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                      .str.replace('\u00A0','',regex=False)
                      .str.replace('%','',regex=False)
                      .str.replace(' ','',regex=False))
       df[c] = pd.to_numeric(df[c], errors='coerce')
# 6) Checar integridade
print("Dtypes:\n", df[num_cols].dtypes)
print("\nNaN por coluna:\n", df[num_cols].isna().sum())
assert pd.isna(df.loc[len(df)-1, 'close_plus1']), "A última linha (dez/2023) deve estar vazia em close_plus1."
# 7) Treino
FEATS  = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
train = df.dropna(subset=FEATS + [TARGET]).copy()
print("\nLinhas totais:", len(df), " | Linhas após dropna:", len(train))
assert len(train) >= 12, "Poucos dados após limpeza."
X = train[FEATS].values
y = train[TARGET].values
# 8) Modelo Ridge + validação temporal
tscv  = TimeSeriesSplit(n_splits=5)
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# 9) Validação out-of-fold
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
mse  = mean_squared_error(y[mask], y_oof[mask])
rmse = np.sqrt(mse)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge  | MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# 10) Projeção recursiva JAN–DEZ/2024
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
work = df[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
rows = []
for _ in range(12):  # 12 meses de 2024
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   next_date = work['date'].iloc[-1] + pd.offsets.MonthBegin(1)
   ret_next  = (pred_close_next / last_close) - 1.0
   # anexa linha prevista
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   # recalcula features
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_ridge = pd.DataFrame(rows)
# 11) Salvar em Excel
try:
   with pd.ExcelWriter('bova11_pred_2024_ridge.xlsx', engine='xlsxwriter') as w:
       pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE':mae,'RMSE':rmse,'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
except Exception:
   with pd.ExcelWriter('bova11_pred_2024_ridge.xlsx', engine='openpyxl') as w:
       pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE':mae,'RMSE':rmse,'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
print("\n✅ Arquivo gerado: bova11_pred_2024_ridge.xlsx")

Dtypes:
 close          float64
ret_m          float64
mm3            float64
mm6            float64
mm12           float64
vol3           float64
vol6           float64
momentum3      float64
volrel         float64
close_plus1    float64
dtype: object

NaN por coluna:
 close          0
ret_m          0
mm3            0
mm6            0
mm12           0
vol3           0
vol6           0
momentum3      0
volrel         0
close_plus1    1
dtype: int64

Linhas totais: 48  | Linhas após dropna: 47

Ridge  | MAE=1.910  RMSE=2.494  R²=0.875  best={'m__alpha': 0.1}

✅ Arquivo gerado: bova11_pred_2024_ridge.xlsx


In [7]:
import os
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# =========================================================
# 1) Caminhos / leitura
# =========================================================
INPUT_PATH = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"
BASE_DIR   = os.path.dirname(INPUT_PATH)
OUTPUT_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge.xlsx")
df = pd.read_csv(INPUT_PATH, sep=";", decimal=",", thousands=".")
# =========================================================
# 2) Padronização de colunas / tipos
# =========================================================
df.columns = (df.columns
               .str.strip()
               .str.replace(r"\s+", "_", regex=True)
               .str.lower())
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                      .str.replace('\u00A0','',regex=False)
                      .str.replace('%','',regex=False)
                      .str.replace(' ','',regex=False))
       df[c] = pd.to_numeric(df[c], errors='coerce')
print("Dtypes:\n", df[num_cols].dtypes)
print("\nNaN por coluna:\n", df[num_cols].isna().sum())
assert pd.isna(df.loc[len(df)-1, 'close_plus1']), "A última linha (dez/2023) deve estar vazia em close_plus1."
# =========================================================
# 3) Treino (Ridge) com validação temporal
# =========================================================
FEATS  = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
train = df.dropna(subset=FEATS + [TARGET]).copy()
print("\nLinhas totais:", len(df), " | Linhas após dropna:", len(train))
assert len(train) >= 12, "Poucos dados após limpeza."
X = train[FEATS].values
y = train[TARGET].values
tscv  = TimeSeriesSplit(n_splits=5)
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# OOF com máscara
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
mse  = mean_squared_error(y[mask], y_oof[mask])
rmse = np.sqrt(mse)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge  | MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# =========================================================
# 4) Projeção recursiva JAN–DEZ/2024 (sem macro)
# =========================================================
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
work = df[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
rows = []
for _ in range(12):  # 12 meses de 2024
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   next_date = work['date'].iloc[-1] + pd.offsets.MonthBegin(1)
   ret_next  = (pred_close_next / last_close) - 1.0
   # adiciona linha prevista
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   # recalcula features
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_ridge = pd.DataFrame(rows)
# =========================================================
# 5) Salvar Excel no mesmo diretório do CSV
# =========================================================
try:
   with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as w:
       pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE':mae,'RMSE':rmse,'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
except Exception:
   with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as w:
       pred_ridge.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE':mae,'RMSE':rmse,'R2':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas')
print(f"\n✅ Arquivo gerado em:\n{OUTPUT_PATH}")

Dtypes:
 close          float64
ret_m          float64
mm3            float64
mm6            float64
mm12           float64
vol3           float64
vol6           float64
momentum3      float64
volrel         float64
close_plus1    float64
dtype: object

NaN por coluna:
 close          0
ret_m          0
mm3            0
mm6            0
mm12           0
vol3           0
vol6           0
momentum3      0
volrel         0
close_plus1    1
dtype: int64

Linhas totais: 48  | Linhas após dropna: 47

Ridge  | MAE=1.910  RMSE=2.494  R²=0.875  best={'m__alpha': 0.1}

✅ Arquivo gerado em:
C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_pred_2024_ridge.xlsx


In [8]:
import os
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# =========================================================
# 1) Caminhos / leitura
# =========================================================
INPUT_PATH = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"
BASE_DIR   = os.path.dirname(INPUT_PATH)
OUTPUT_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge.xlsx")
df = pd.read_csv(INPUT_PATH, sep=";", decimal=",", thousands=".")
# =========================================================
# 2) Padronização de colunas / tipos
# =========================================================
df.columns = (df.columns
               .str.strip()
               .str.replace(r"\s+", "_", regex=True)
               .str.lower())
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                      .str.replace('\u00A0','',regex=False)
                      .str.replace('%','',regex=False)
                      .str.replace(' ','',regex=False))
       df[c] = pd.to_numeric(df[c], errors='coerce')
# =========================================================
# 3) Separar histórico (até 2023-12) e, opcionalmente, reais de 2024
#    -> Treino usa SOMENTE df_hist (sem qualquer 2024)
# =========================================================
limite_hist = pd.Timestamp('2023-12-01')  # 1º dia de dez/2023
df_hist = df[df['date'] <= limite_hist].copy()
df_2024 = df[(df['date'] >= pd.Timestamp('2024-01-01')) & (df['date'] <= pd.Timestamp('2024-12-31'))][['date','close']].copy()
df_2024.rename(columns={'close':'close_real'}, inplace=True)
# Se sua base é “até 2023”, a assert abaixo garante que a última linha é a que vamos prever (jan/24)
if 'close_plus1' in df_hist.columns:
   assert pd.isna(df_hist.loc[len(df_hist)-1, 'close_plus1']), "A última linha de 2023 deve estar vazia em close_plus1 (alvo desconhecido)."
print("Linhas histórico (<=2023-12):", len(df_hist))
print("Linhas 2024 (se houver no CSV):", len(df_2024))
# =========================================================
# 4) Treino (Ridge) com validação temporal (somente 2020–2023)
# =========================================================
FEATS  = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
train = df_hist.dropna(subset=FEATS + [TARGET]).copy()
print("Linhas após dropna (treino):", len(train))
assert len(train) >= 12, "Poucos dados após limpeza."
X = train[FEATS].values
y = train[TARGET].values
tscv  = TimeSeriesSplit(n_splits=5)
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# OOF com máscara (métricas apenas no histórico)
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
mse  = mean_squared_error(y[mask], y_oof[mask])
rmse = np.sqrt(mse)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge  | (Histórico 2020–2023)  MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# =========================================================
# 5) Projeção recursiva: gerar exatamente 12 datas de 2024 (sem usar 2024 real)
# =========================================================
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
# base de trabalho começa no histórico (até 2023-12)
work = df_hist[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
alvos_2024 = pd.date_range('2024-01-01', '2024-12-01', freq='MS')  # 12 meses fixos
rows = []
for next_date in alvos_2024:
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3  # hipótese para o mês seguinte
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   ret_next  = (pred_close_next / last_close) - 1.0
   # anexa linha prevista
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   # recalcula features na nova última linha
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_2024 = pd.DataFrame(rows).sort_values('date').reset_index(drop=True)
# =========================================================
# 6) (Opcional) Avaliar contra reais de 2024, se existirem no CSV
#     -> NÃO são usados no treino; apenas comparação em holdout
# =========================================================
avaliacao_2024 = None
if len(df_2024) > 0:
   avaliacao_2024 = pred_2024.merge(df_2024, on='date', how='left')
   if 'close_real' in avaliacao_2024.columns and avaliacao_2024['close_real'].notna().any():
       avaliacao_2024['erro_abs'] = (avaliacao_2024['close_pred'] - avaliacao_2024['close_real']).abs()
       avaliacao_2024['erro_%']   = (avaliacao_2024['close_pred'] / avaliacao_2024['close_real'] - 1.0) * 100
       mae_24  = avaliacao_2024['erro_abs'].mean(skipna=True)
       rmse_24 = np.sqrt(((avaliacao_2024['close_pred'] - avaliacao_2024['close_real'])**2).mean(skipna=True))
       print(f"\n(Comparação holdout 2024) MAE={mae_24:.3f}  RMSE={rmse_24:.3f}  (onde houver real)")
   else:
       print("\nSem closes reais de 2024 no CSV; apenas previsões foram geradas.")
# =========================================================
# 7) Salvar Excel no mesmo diretório do CSV
# =========================================================
try:
   with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as w:
       pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')  # só 2024
       pd.DataFrame([{'MAE_hist':mae,'RMSE_hist':rmse,'R2_hist':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas_hist')
       if avaliacao_2024 is not None:
           avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
except Exception:
   with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as w:
       pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE_hist':mae,'RMSE_hist':rmse,'R2_hist':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas_hist')
       if avaliacao_2024 is not None:
           avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
print(f"\n✅ Arquivo gerado em:\n{OUTPUT_PATH}")

Linhas histórico (<=2023-12): 48
Linhas 2024 (se houver no CSV): 0
Linhas após dropna (treino): 47

Ridge  | (Histórico 2020–2023)  MAE=1.910  RMSE=2.494  R²=0.875  best={'m__alpha': 0.1}

✅ Arquivo gerado em:
C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_pred_2024_ridge.xlsx


In [9]:
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
# =========================================================
# Caminhos dos arquivos
# =========================================================
BASE_DIR = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11"
PRED_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge.xlsx")      # arquivo do modelo
REAL_PATH = os.path.join(BASE_DIR, "bova11_real_2024.csv")             # seu arquivo com dados reais
# =========================================================
# Leitura
# =========================================================
pred = pd.read_excel(PRED_PATH, sheet_name='predicoes_2024')
real = pd.read_csv(REAL_PATH, sep=";", decimal=",", thousands=".")
# Ajusta formatos
pred['date'] = pd.to_datetime(pred['date'])
real.columns = real.columns.str.strip().str.lower()
real['date'] = pd.to_datetime(real['date'], dayfirst=True, errors='coerce')
real = real.rename(columns={'ultimo':'close','último':'close','fechamento':'close'})
# =========================================================
# Merge (junção real + previsto)
# =========================================================
comp = pd.merge(pred, real[['date','close']], on='date', how='inner')
comp.rename(columns={'close':'close_real','close_pred':'close_pred'}, inplace=True)
comp['erro_abs'] = (comp['close_pred'] - comp['close_real']).abs()
comp['erro_%']   = (comp['close_pred']/comp['close_real'] - 1)*100
# =========================================================
# Métricas
# =========================================================
mae  = mean_absolute_error(comp['close_real'], comp['close_pred'])
rmse = np.sqrt(mean_squared_error(comp['close_real'], comp['close_pred']))
r2   = r2_score(comp['close_real'], comp['close_pred'])
print("Comparação 2024:")
print(f"MAE = {mae:.3f}  |  RMSE = {rmse:.3f}  |  R² = {r2:.3f}")
# =========================================================
# 2️⃣ Gráfico real vs previsto
# =========================================================
plt.figure(figsize=(10,5))
plt.plot(comp['date'], comp['close_real'], label='Real 2024', linewidth=2)
plt.plot(comp['date'], comp['close_pred'], label='Previsto 2024 (modelo)', linewidth=2, linestyle='--')
plt.title('Comparativo: BOVA11 Real vs Previsto (2024)')
plt.xlabel('Data')
plt.ylabel('Preço de Fechamento')
plt.legend()
plt.grid(True)
plt.show()
# =========================================================
# 3️⃣ Exportar resultados para Excel
# =========================================================
OUT_PATH = os.path.join(BASE_DIR, "avaliacao_2024_ridge.xlsx")
with pd.ExcelWriter(OUT_PATH, engine='xlsxwriter') as w:
   comp.to_excel(w, index=False, sheet_name='comparativo')
   pd.DataFrame([{'MAE':mae, 'RMSE':rmse, 'R2':r2}]).to_excel(w, index=False, sheet_name='metricas')
print(f"\n✅ Avaliação salva em:\n{OUT_PATH}")

Matplotlib is building the font cache; this may take a moment.


KeyError: 'date'

In [10]:
import os
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# =========================================================
# 1) Caminhos / leitura
# =========================================================
INPUT_PATH = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"
BASE_DIR   = os.path.dirname(INPUT_PATH)
OUTPUT_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge.xlsx")
df = pd.read_csv(INPUT_PATH, sep=";", decimal=",", thousands=".")
# =========================================================
# 2) Padronização de colunas / tipos
# =========================================================
df.columns = (df.columns
               .str.strip()
               .str.replace(r"\s+", "_", regex=True)
               .str.lower())
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                      .str.replace('\u00A0','',regex=False)
                      .str.replace('%','',regex=False)
                      .str.replace(' ','',regex=False))
       df[c] = pd.to_numeric(df[c], errors='coerce')
# =========================================================
# 3) Separar histórico (até 2023-12) e, opcionalmente, reais de 2024
#    -> Treino usa SOMENTE df_hist (sem qualquer 2024)
# =========================================================
limite_hist = pd.Timestamp('2023-12-01')  # 1º dia de dez/2023
df_hist = df[df['date'] <= limite_hist].copy()
df_2024 = df[(df['date'] >= pd.Timestamp('2024-01-01')) & (df['date'] <= pd.Timestamp('2024-12-31'))][['date','close']].copy()
df_2024.rename(columns={'close':'close_real'}, inplace=True)
# Se sua base é “até 2023”, a assert abaixo garante que a última linha é a que vamos prever (jan/24)
if 'close_plus1' in df_hist.columns:
   assert pd.isna(df_hist.loc[len(df_hist)-1, 'close_plus1']), "A última linha de 2023 deve estar vazia em close_plus1 (alvo desconhecido)."
print("Linhas histórico (<=2023-12):", len(df_hist))
print("Linhas 2024 (se houver no CSV):", len(df_2024))
# =========================================================
# 4) Treino (Ridge) com validação temporal (somente 2020–2023)
# =========================================================
FEATS  = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
train = df_hist.dropna(subset=FEATS + [TARGET]).copy()
print("Linhas após dropna (treino):", len(train))
assert len(train) >= 12, "Poucos dados após limpeza."
X = train[FEATS].values
y = train[TARGET].values
tscv  = TimeSeriesSplit(n_splits=5)
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# OOF com máscara (métricas apenas no histórico)
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
mse  = mean_squared_error(y[mask], y_oof[mask])
rmse = np.sqrt(mse)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge  | (Histórico 2020–2023)  MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.3f}  best={g.best_params_}")
# =========================================================
# 5) Projeção recursiva: gerar exatamente 12 datas de 2024 (sem usar 2024 real)
# =========================================================
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
# base de trabalho começa no histórico (até 2023-12)
work = df_hist[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
alvos_2024 = pd.date_range('2024-01-01', '2024-12-01', freq='MS')  # 12 meses fixos
rows = []
for next_date in alvos_2024:
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3  # hipótese para o mês seguinte
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   ret_next  = (pred_close_next / last_close) - 1.0
   # anexa linha prevista
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date, 'close': pred_close_next, 'ret_m': ret_next, 'volrel': volrel_last3
   }])], ignore_index=True)
   # recalcula features na nova última linha
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_2024 = pd.DataFrame(rows).sort_values('date').reset_index(drop=True)
# =========================================================
# 6) (Opcional) Avaliar contra reais de 2024, se existirem no CSV
#     -> NÃO são usados no treino; apenas comparação em holdout
# =========================================================
avaliacao_2024 = None
if len(df_2024) > 0:
   avaliacao_2024 = pred_2024.merge(df_2024, on='date', how='left')
   if 'close_real' in avaliacao_2024.columns and avaliacao_2024['close_real'].notna().any():
       avaliacao_2024['erro_abs'] = (avaliacao_2024['close_pred'] - avaliacao_2024['close_real']).abs()
       avaliacao_2024['erro_%']   = (avaliacao_2024['close_pred'] / avaliacao_2024['close_real'] - 1.0) * 100
       mae_24  = avaliacao_2024['erro_abs'].mean(skipna=True)
       rmse_24 = np.sqrt(((avaliacao_2024['close_pred'] - avaliacao_2024['close_real'])**2).mean(skipna=True))
       print(f"\n(Comparação holdout 2024) MAE={mae_24:.3f}  RMSE={rmse_24:.3f}  (onde houver real)")
   else:
       print("\nSem closes reais de 2024 no CSV; apenas previsões foram geradas.")
# =========================================================
# 7) Salvar Excel no mesmo diretório do CSV
# =========================================================
try:
   with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as w:
       pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')  # só 2024
       pd.DataFrame([{'MAE_hist':mae,'RMSE_hist':rmse,'R2_hist':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas_hist')
       if avaliacao_2024 is not None:
           avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
except Exception:
   with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as w:
       pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE_hist':mae,'RMSE_hist':rmse,'R2_hist':r2, **g.best_params_}]).to_excel(w, index=False, sheet_name='metricas_hist')
       if avaliacao_2024 is not None:
           avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
print(f"\n✅ Arquivo gerado em:\n{OUTPUT_PATH}")

Linhas histórico (<=2023-12): 48
Linhas 2024 (se houver no CSV): 0
Linhas após dropna (treino): 47

Ridge  | (Histórico 2020–2023)  MAE=1.910  RMSE=2.494  R²=0.875  best={'m__alpha': 0.1}

✅ Arquivo gerado em:
C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_pred_2024_ridge.xlsx


In [11]:
# =========================================================
# 1) IMPORTS
# =========================================================
import os
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# =========================================================
# 2) CAMINHOS / LEITURA
# =========================================================
INPUT_PATH = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"
BASE_DIR = os.path.dirname(INPUT_PATH)
OUTPUT_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge.xlsx")
df = pd.read_csv(INPUT_PATH, sep=";", decimal=",", thousands=".")
# =========================================================
# 3) PADRONIZAÇÃO DE COLUNAS / TIPOS
# =========================================================
df.columns = (df.columns
   .str.strip()
   .str.replace(r"\s+", "_", regex=True)
   .str.lower()
)
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
num_cols = ['close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','close_plus1']
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                       .str.replace('\u00A0','',regex=False)
                       .str.replace('%','',regex=False)
                       .str.replace(' ','',regex=False))
       df[c] = pd.to_numeric(df[c], errors='coerce')
# =========================================================
# 4) TRATAMENTO CASO close_plus1 NÃO EXISTA
# =========================================================
if 'close_plus1' not in df.columns:
   df['close_plus1'] = df['close'].shift(-1)
# =========================================================
# 5) SEPARAR HISTÓRICO (até 2023-12) e REAIS DE 2024 (se houver)
# =========================================================
limite_hist = pd.Timestamp('2023-12-01')
df_hist = df[df['date'] <= limite_hist].copy()
df_2024 = df[(df['date'] >= pd.Timestamp('2024-01-01')) & (df['date'] <= pd.Timestamp('2024-12-31'))][['date','close']].copy()
df_2024.rename(columns={'close':'close_real'}, inplace=True)
if 'close_plus1' in df_hist.columns:
   assert pd.isna(df_hist.loc[len(df_hist)-1, 'close_plus1']), \
       "A última linha de 2023 deve estar vazia em close_plus1 (alvo desconhecido)."
print("Linhas histórico (<=2023-12):", len(df_hist))
print("Linhas 2024 (se houver no CSV):", len(df_2024))
# =========================================================
# 6) TREINO (RIDGE) COM VALIDAÇÃO TEMPORAL
# =========================================================
FEATS = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
TARGET = 'close_plus1'
train = df_hist.dropna(subset=FEATS + [TARGET]).copy()
print("Linhas após dropna (treino):", len(train))
assert len(train) >= 12, "Poucos dados após limpeza."
X = train[FEATS].values
y = train[TARGET].values
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
tscv  = TimeSeriesSplit(n_splits=5)
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# =========================================================
# 7) OUT-OF-FOLD MÉTRICAS (HISTÓRICO)
# =========================================================
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
mse  = mean_squared_error(y[mask], y_oof[mask])
rmse = np.sqrt(mse)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge | (Histórico 2020–2023) MAE={mae:.3f} RMSE={rmse:.3f} R²={r2:.3f} best={g.best_params_}")
# =========================================================
# 8) REFIT NO HISTÓRICO COMPLETO (CORREÇÃO IMPORTANTE)
# =========================================================
best_ridge.fit(X, y)
# =========================================================
# 9) PROJEÇÃO RECURSIVA DE 12 MESES (2024)
# =========================================================
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
work = df_hist[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
rows = []
alvos_2024 = pd.date_range('2024-01-01', '2024-12-01', freq='MS')
for next_date in alvos_2024:
   feats_now = work.iloc[-1][FEATS].to_dict()
   feats_now['volrel'] = volrel_last3
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   ret_next  = (pred_close_next / last_close) - 1.0
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date,
       'close': pred_close_next,
       'ret_m': ret_next,
       'volrel': volrel_last3
   }])], ignore_index=True)
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes, 12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_2024 = pd.DataFrame(rows).sort_values('date').reset_index(drop=True)
# =========================================================
# 10) AVALIAÇÃO CONTRA REAIS DE 2024 (SE EXISTIREM)
# =========================================================
avaliacao_2024 = None
if len(df_2024) > 0:
   avaliacao_2024 = pred_2024.merge(df_2024, on='date', how='left')
   if 'close_real' in avaliacao_2024.columns and avaliacao_2024['close_real'].notna().any():
       avaliacao_2024['erro_abs'] = (avaliacao_2024['close_pred'] - avaliacao_2024['close_real']).abs()
       avaliacao_2024['erro_%']   = (avaliacao_2024['close_pred'] / avaliacao_2024['close_real'] - 1.0) * 100
       mae_24  = avaliacao_2024['erro_abs'].mean(skipna=True)
       rmse_24 = np.sqrt(((avaliacao_2024['close_pred'] - avaliacao_2024['close_real'])**2).mean(skipna=True))
       print(f"\n(Comparação holdout 2024) MAE={mae_24:.3f} RMSE={rmse_24:.3f}")
   else:
       print("\nSem closes reais de 2024 no CSV; apenas previsões foram geradas.")
# =========================================================
# 11) SALVAR RESULTADOS NO EXCEL
# =========================================================
try:
   with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as w:
       pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE_hist': mae, 'RMSE_hist': rmse, 'R2_hist': r2, **g.best_params_}]).to_excel(
           w, index=False, sheet_name='metricas_hist')
       if avaliacao_2024 is not None:
           avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
except Exception:
   with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as w:
       pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
       pd.DataFrame([{'MAE_hist': mae, 'RMSE_hist': rmse, 'R2_hist': r2, **g.best_params_}]).to_excel(
           w, index=False, sheet_name='metricas_hist')
       if avaliacao_2024 is not None:
           avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
print(f"\n✅ Arquivo gerado com sucesso em:\n{OUTPUT_PATH}")

Linhas histórico (<=2023-12): 48
Linhas 2024 (se houver no CSV): 0
Linhas após dropna (treino): 47

Ridge | (Histórico 2020–2023) MAE=1.910 RMSE=2.494 R²=0.875 best={'m__alpha': 0.1}

✅ Arquivo gerado com sucesso em:
C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_pred_2024_ridge.xlsx


In [12]:
# =========================================================
# 📘 PREVISÃO BOVA11 COM REGRESSÃO RIDGE
#  - Usa features internas + externas (selic_m, ipca_m)
#  - Treino: 2020–2023 | Previsão: jan–dez/2024 sem usar 2024 real
# =========================================================
import os
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# ======= CONFIG RÁPIDA =======
INPUT_PATH = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"
BASE_DIR   = os.path.dirname(INPUT_PATH)
OUTPUT_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge_macro.xlsx")
# Cenário para 2024 (como preencher selic/ipca no futuro sem usar 2024 real)
MACRO_METHOD = "mean3"   # opções: "last" (último valor), "mean3" (média últimos 3), "mean6" (média últimos 6)
# =========================================================
# 1) Leitura e padronização
# =========================================================
df = pd.read_csv(INPUT_PATH, sep=";", decimal=",", thousands=".")
df.columns = (df.columns
             .str.strip()
             .str.replace(r"\s+", "_", regex=True)
             .str.lower())
rename_map = {
   'data': 'date',
   'ultimo': 'close', 'último': 'close',
   'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
   'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
   'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
   'média_móvel_12m': 'mm12', 'media_movel_12m': 'mm12',
   'volatilidade_3m': 'vol3', 'vol3': 'vol3',
   'volatilidade_6m': 'vol6', 'vol6': 'vol6',
   'momentum': 'momentum3', 'momentum3': 'momentum3',
   'volume_relativo': 'volrel', 'volrel': 'volrel',
   'close_plus1': 'close_plus1', 'close_+1': 'close_plus1',
   # nomes alternativos para macro, se vierem diferentes
   'selic': 'selic_m', 'selic_%': 'selic_m',
   'ipca': 'ipca_m', 'ipca_%': 'ipca_m'
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
# Converte para numérico removendo símbolos comuns
num_cols = [
   'close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel',
   'close_plus1','selic_m','ipca_m'
]
for c in num_cols:
   if c in df.columns:
       df[c] = (df[c].astype(str)
                  .str.replace('\u00A0','',regex=False)  # espaço não separável
                  .str.replace('%','',regex=False)
                  .str.replace(' ','',regex=False))
       df[c] = pd.to_numeric(df[c], errors='coerce')
# Se faltar close_plus1, cria via shift(-1)
if 'close_plus1' not in df.columns:
   df['close_plus1'] = df['close'].shift(-1)
# Checa presença das variáveis macro
assert 'selic_m' in df.columns, "Coluna 'selic_m' não encontrada no CSV."
assert 'ipca_m' in df.columns,  "Coluna 'ipca_m' não encontrada no CSV."
# =========================================================
# 2) Separa histórico (<= 2023-12) e 2024 real (opcional p/ comparação)
# =========================================================
limite_hist = pd.Timestamp('2023-12-01')
df_hist = df[df['date'] <= limite_hist].copy()
df_2024 = df[(df['date'] >= pd.Timestamp('2024-01-01')) &
            (df['date'] <= pd.Timestamp('2024-12-31'))][['date','close']].copy()
df_2024.rename(columns={'close':'close_real'}, inplace=True)
# Última linha do histórico deve ter alvo desconhecido (por segurança, se existir)
if 'close_plus1' in df_hist.columns:
   assert pd.isna(df_hist.loc[len(df_hist)-1, 'close_plus1']), \
       "A última linha de 2023 deve estar vazia em close_plus1 (alvo desconhecido)."
print("Linhas histórico (<=2023-12):", len(df_hist))
print("Linhas 2024 (se houver no CSV):", len(df_2024))
# =========================================================
# 3) Treino (Ridge) com validação temporal
# =========================================================
FEATS_BASE  = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel']
FEATS_MACRO = ['selic_m','ipca_m']
FEATS       = FEATS_BASE + FEATS_MACRO   # agora incluem as externas
TARGET = 'close_plus1'
train = df_hist.dropna(subset=FEATS + [TARGET]).copy()
print("Linhas após dropna (treino):", len(train))
assert len(train) >= 12, "Poucos dados após limpeza (verifique NaNs em selic_m/ipca_m)."
X = train[FEATS].values
y = train[TARGET].values
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
tscv  = TimeSeriesSplit(n_splits=5)
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# Métricas OOF
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
   best_ridge.fit(X[tr], y[tr])
   y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
rmse = mean_squared_error(y[mask], y_oof[mask], squared=False)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRIDGE (com macro) | MAE={mae:.3f} RMSE={rmse:.3f} R²={r2:.3f} best={g.best_params_}")
# Refit no histórico completo
best_ridge.fit(X, y)
# =========================================================
# 4) Projeção recursiva 2024 (preenche selic/ipca conforme cenário escolhido)
# =========================================================
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
def macro_future_value(series: pd.Series, method: str):
   s = series.dropna().values
   if len(s) == 0:
       return np.nan
   if method == "last":
       return float(s[-1])
   elif method == "mean3":
       return float(np.mean(s[-3:])) if len(s) >= 3 else float(np.mean(s))
   elif method == "mean6":
       return float(np.mean(s[-6:])) if len(s) >= 6 else float(np.mean(s))
   else:
       # fallback: último
       return float(s[-1])
work_cols = ['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','selic_m','ipca_m']
work = df_hist[work_cols].copy()
# Cenários fixos para 2024
selic_future = macro_future_value(work['selic_m'], MACRO_METHOD)
ipca_future  = macro_future_value(work['ipca_m'],  MACRO_METHOD)
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
alvos_2024 = pd.date_range('2024-01-01', '2024-12-01', freq='MS')
rows = []
for next_date in alvos_2024:
   feats_now = work.iloc[-1][FEATS].to_dict()
   # Impõe valores "macro" futuros conforme cenário (sem usar 2024 real)
   feats_now['selic_m'] = selic_future
   feats_now['ipca_m']  = ipca_future
   # Para volrel também usamos cenário simples (média dos 3 últimos)
   feats_now['volrel']  = volrel_last3
   # Prever close do próximo mês
   x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
   pred_close_next = float(best_ridge.predict(x)[0])
   # Atualiza retorno e adiciona linha prevista
   ret_next = (pred_close_next / last_close) - 1.0
   work = pd.concat([work, pd.DataFrame([{
       'date': next_date,
       'close': pred_close_next,
       'ret_m': ret_next,
       'volrel': volrel_last3,
       'selic_m': selic_future,
       'ipca_m':  ipca_future
   }])], ignore_index=True)
   # Recalcula features de média/vol/ momentum
   closes = work['close'].values
   rets   = work['ret_m'].values
   work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
   work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
   work.loc[work.index[-1], 'mm12'] = roll_mean(closes,12)
   work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
   work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
   if len(closes) >= 4:
       work.loc[work.index[-1], 'momentum3'] = (closes[-1] / closes[-4]) - 1
   last_close = pred_close_next
   rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_2024 = pd.DataFrame(rows).sort_values('date').reset_index(drop=True)
# =========================================================
# 5) (Opcional) Comparar com closes reais de 2024 (se estiverem no CSV)
# =========================================================
avaliacao_2024 = None
if len(df_2024) > 0:
   avaliacao_2024 = pred_2024.merge(df_2024, on='date', how='left')
   if 'close_real' in avaliacao_2024.columns and avaliacao_2024['close_real'].notna().any():
       avaliacao_2024['erro_abs'] = (avaliacao_2024['close_pred'] - avaliacao_2024['close_real']).abs()
       avaliacao_2024['erro_%']   = (avaliacao_2024['close_pred'] / avaliacao_2024['close_real'] - 1.0) * 100
       mae_24  = avaliacao_2024['erro_abs'].mean(skipna=True)
       rmse_24 = np.sqrt(((avaliacao_2024['close_pred'] - avaliacao_2024['close_real'])**2).mean(skipna=True))
       print(f"\n(Comparação holdout 2024) MAE={mae_24:.3f} RMSE={rmse_24:.3f}")
   else:
       print("\nSem closes reais de 2024 no CSV; apenas previsões foram geradas.")
# =========================================================
# 6) Exporta para Excel (mesmo diretório do CSV)
# =========================================================
with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as w:
   # Previsões 2024
   pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
   # Métricas do histórico
   pd.DataFrame([{
       'MAE_hist': mae, 'RMSE_hist': rmse, 'R2_hist': r2, **g.best_params_
   }]).to_excel(w, index=False, sheet_name='metricas_hist')
   # Avaliação 2024 (se houver real)
   if avaliacao_2024 is not None:
       avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
   # Documentação do cenário usado para macro
   doc = pd.DataFrame([{
       'cenario_macro': MACRO_METHOD,
       'selic_2024_usada': selic_future,
       'ipca_2024_usado': ipca_future,
       'volrel_2024_usado': float(volrel_last3)
   }])
   doc.to_excel(w, index=False, sheet_name='cenario_macro_2024')
print(f"\n✅ Arquivo gerado com sucesso em:\n{OUTPUT_PATH}")

Linhas histórico (<=2023-12): 48
Linhas 2024 (se houver no CSV): 0
Linhas após dropna (treino): 47


TypeError: got an unexpected keyword argument 'squared'

In [13]:
# =========================================================
# 1) IMPORTS
# =========================================================
import os
import pandas as pd, numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# =========================================================
# 2) CAMINHOS / LEITURA
# =========================================================
INPUT_PATH = r"C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_ate_2023.csv"
BASE_DIR = os.path.dirname(INPUT_PATH)
OUTPUT_PATH = os.path.join(BASE_DIR, "bova11_pred_2024_ridge.xlsx")
df = pd.read_csv(INPUT_PATH, sep=";", decimal=",", thousands=".")
# =========================================================
# 3) PADRONIZAÇÃO DE COLUNAS / TIPOS
# =========================================================
df.columns = (df.columns
  .str.strip()
  .str.replace(r"\s+", "_", regex=True)
  .str.lower()
)
rename_map = {
  'data': 'date',
  'ultimo': 'close', 'último': 'close',
  'retorno_mensal_(%)': 'ret_m', 'retorno_mensal': 'ret_m',
  'média_móvel_3m': 'mm3', 'media_movel_3m': 'mm3',
  'média_móvel_6m': 'mm6', 'media_movel_6m': 'mm6',
  'média_móvel_12m': 'mm12','media_movel_12m': 'mm12',
  'volatilidade_3m': 'vol3', 'vol3': 'vol3',
  'volatilidade_6m': 'vol6', 'vol6': 'vol6',
  'momentum': 'momentum3', 'momentum3': 'momentum3',
  'volume_relativo': 'volrel', 'volrel': 'volrel',
  'close_plus1': 'close_plus1', 'close_+1': 'close_plus1'
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
# >>>>>>> ADIÇÃO: garante que existam as colunas macro selic_m e ipca_m <<<<<<<
# (Se já existirem com esses nomes, nada muda)
# Caso venham com %, vírgula etc., o tratamento abaixo limpa.
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df = df.sort_values('date').reset_index(drop=True)
num_cols = [
   'close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel',
   'close_plus1','selic_m','ipca_m'   # << ADIÇÃO
]
for c in num_cols:
  if c in df.columns:
      df[c] = (df[c].astype(str)
                      .str.replace('\u00A0','',regex=False)
                      .str.replace('%','',regex=False)
                      .str.replace(' ','',regex=False))
      df[c] = pd.to_numeric(df[c], errors='coerce')
# =========================================================
# 4) TRATAMENTO CASO close_plus1 NÃO EXISTA
# =========================================================
if 'close_plus1' not in df.columns:
  df['close_plus1'] = df['close'].shift(-1)
# =========================================================
# 5) SEPARAR HISTÓRICO (até 2023-12) e REAIS DE 2024 (se houver)
# =========================================================
limite_hist = pd.Timestamp('2023-12-01')
df_hist = df[df['date'] <= limite_hist].copy()
df_2024 = df[(df['date'] >= pd.Timestamp('2024-01-01')) & (df['date'] <= pd.Timestamp('2024-12-31'))][['date','close']].copy()
df_2024.rename(columns={'close':'close_real'}, inplace=True)
# Segurança: última linha de 2023 sem alvo conhecido
if 'close_plus1' in df_hist.columns:
  assert pd.isna(df_hist.loc[len(df_hist)-1, 'close_plus1']), \
      "A última linha de 2023 deve estar vazia em close_plus1 (alvo desconhecido)."
print("Linhas histórico (<=2023-12):", len(df_hist))
print("Linhas 2024 (se houver no CSV):", len(df_2024))
# =========================================================
# 6) TREINO (RIDGE) COM VALIDAÇÃO TEMPORAL
# =========================================================
# >>>>>>> ADIÇÃO: inclui selic_m e ipca_m nas features <<<<<<<
FEATS = ['ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','selic_m','ipca_m']
TARGET = 'close_plus1'
train = df_hist.dropna(subset=FEATS + [TARGET]).copy()
print("Linhas após dropna (treino):", len(train))
assert len(train) >= 12, "Poucos dados após limpeza (verifique NaNs em selic_m/ipca_m)."
X = train[FEATS].values
y = train[TARGET].values
ridge = Pipeline([('scaler', StandardScaler()), ('m', Ridge())])
grid  = {'m__alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
tscv  = TimeSeriesSplit(n_splits=5)
g = GridSearchCV(ridge, grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
g.fit(X, y)
best_ridge = g.best_estimator_
# =========================================================
# 7) OUT-OF-FOLD MÉTRICAS (HISTÓRICO)
# =========================================================
y_oof = np.full_like(y, np.nan, dtype=float)
for tr, te in tscv.split(X):
  best_ridge.fit(X[tr], y[tr])
  y_oof[te] = best_ridge.predict(X[te])
mask = ~np.isnan(y_oof)
mae  = mean_absolute_error(y[mask], y_oof[mask])
mse  = mean_squared_error(y[mask], y_oof[mask])
rmse = np.sqrt(mse)
r2   = r2_score(y[mask], y_oof[mask])
print(f"\nRidge | (Histórico 2020–2023) MAE={mae:.3f} RMSE={rmse:.3f} R²={r2:.3f} best={g.best_params_}")
# =========================================================
# 8) REFIT NO HISTÓRICO COMPLETO (CORREÇÃO IMPORTANTE)
# =========================================================
best_ridge.fit(X, y)
# =========================================================
# 9) PROJEÇÃO RECURSIVA DE 12 MESES (2024)
# =========================================================
def roll_mean(a,n): return np.mean(a[-n:]) if len(a)>=n else np.nan
def roll_std(a,n):  return np.std(a[-n:], ddof=1) if len(a)>=n else np.nan
# >>>>>>> ADIÇÃO: manter selic/ipca fixos em 2024 (último valor de 2023) <<<<<<<
selic_last = df_hist['selic_m'].dropna().iloc[-1] if 'selic_m' in df_hist.columns else np.nan
ipca_last  = df_hist['ipca_m'].dropna().iloc[-1]  if 'ipca_m'  in df_hist.columns else np.nan
work = df_hist[['date','close','ret_m','mm3','mm6','mm12','vol3','vol6','momentum3','volrel','selic_m','ipca_m']].copy()
volrel_last3 = work['volrel'].tail(3).mean()
last_close   = work['close'].iloc[-1]
rows = []
alvos_2024 = pd.date_range('2024-01-01', '2024-12-01', freq='MS')
for next_date in alvos_2024:
  feats_now = work.iloc[-1][FEATS].to_dict()
  # fixa macro para 2024 (sem usar dados reais de 2024)
  feats_now['selic_m'] = selic_last
  feats_now['ipca_m']  = ipca_last
  # mantém volrel simples (média 3 últimos)
  feats_now['volrel']  = volrel_last3
  x = np.array([feats_now[f] for f in FEATS], dtype=float).reshape(1, -1)
  pred_close_next = float(best_ridge.predict(x)[0])
  ret_next = (pred_close_next / last_close) - 1.0
  work = pd.concat([work, pd.DataFrame([{
      'date': next_date,
      'close': pred_close_next,
      'ret_m': ret_next,
      'volrel': volrel_last3,
      'selic_m': selic_last,
      'ipca_m': ipca_last
  }])], ignore_index=True)
  closes = work['close'].values
  rets   = work['ret_m'].values
  work.loc[work.index[-1], 'mm3']  = roll_mean(closes, 3)
  work.loc[work.index[-1], 'mm6']  = roll_mean(closes, 6)
  work.loc[work.index[-1], 'mm12'] = roll_mean(closes, 12)
  work.loc[work.index[-1], 'vol3'] = roll_std(rets, 3)
  work.loc[work.index[-1], 'vol6'] = roll_std(rets, 6)
  if len(closes) >= 4:
      work.loc[work.index[-1], 'momentum3'] = (closes[-1]/closes[-4]) - 1
  last_close = pred_close_next
  rows.append({'date': next_date, 'close_pred': pred_close_next})
pred_2024 = pd.DataFrame(rows).sort_values('date').reset_index(drop=True)
# =========================================================
# 10) AVALIAÇÃO CONTRA REAIS DE 2024 (SE EXISTIREM)
# =========================================================
avaliacao_2024 = None
if len(df_2024) > 0:
  avaliacao_2024 = pred_2024.merge(df_2024, on='date', how='left')
  if 'close_real' in avaliacao_2024.columns and avaliacao_2024['close_real'].notna().any():
      avaliacao_2024['erro_abs'] = (avaliacao_2024['close_pred'] - avaliacao_2024['close_real']).abs()
      avaliacao_2024['erro_%']   = (avaliacao_2024['close_pred'] / avaliacao_2024['close_real'] - 1.0) * 100
      mae_24  = avaliacao_2024['erro_abs'].mean(skipna=True)
      rmse_24 = np.sqrt(((avaliacao_2024['close_pred'] - avaliacao_2024['close_real'])**2).mean(skipna=True))
      print(f"\n(Comparação holdout 2024) MAE={mae_24:.3f} RMSE={rmse_24:.3f}")
  else:
      print("\nSem closes reais de 2024 no CSV; apenas previsões foram geradas.")
# =========================================================
# 11) SALVAR RESULTADOS NO EXCEL
# =========================================================
try:
  with pd.ExcelWriter(OUTPUT_PATH, engine='xlsxwriter') as w:
      pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
      pd.DataFrame([{'MAE_hist': mae, 'RMSE_hist': rmse, 'R2_hist': r2, **g.best_params_}]).to_excel(
          w, index=False, sheet_name='metricas_hist')
      if avaliacao_2024 is not None:
          avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
except Exception:
  with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as w:
      pred_2024.to_excel(w, index=False, sheet_name='predicoes_2024')
      pd.DataFrame([{'MAE_hist': mae, 'RMSE_hist': rmse, 'R2_hist': r2, **g.best_params_}]).to_excel(
          w, index=False, sheet_name='metricas_hist')
      if avaliacao_2024 is not None:
          avaliacao_2024.to_excel(w, index=False, sheet_name='avaliacao_2024')
print(f"\n✅ Arquivo gerado com sucesso em:\n{OUTPUT_PATH}")

Linhas histórico (<=2023-12): 48
Linhas 2024 (se houver no CSV): 0
Linhas após dropna (treino): 47

Ridge | (Histórico 2020–2023) MAE=1.771 RMSE=2.149 R²=0.907 best={'m__alpha': 1.0}

✅ Arquivo gerado com sucesso em:
C:\Users\kinagaki\OneDrive - Digicorner\Desktop\Trabalho de Conclusão de Curso - UFRJ.2025\Correção\versão atualizada\TESTE BOVA 11\bova11_pred_2024_ridge.xlsx
